## Hyperparameter Search with Simulated Annealing (Optuna)

This example demonstrates how to perform hyperparameter search for a Simulated Annealing training setup using **Optuna** and pyperch’s search utilities.  

A fixed base training configuration is defined, and Optuna is used to explore SA-specific hyperparameters such as temperature, cooling rate, step size, and training duration.  The objective is to maximize validation accuracy on a binary classification task.

In [4]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from pyperch import Trainer
from pyperch.config import TrainConfig, OptimizerConfig
from pyperch.core.metrics import Accuracy
from pyperch.models.mlp import SimpleMLP
from pyperch.core.callbacks import EarlyStopping, CallbackList

# Search components
from pyperch.search.strategy import OptunaStrategy
from pyperch.search.builder import TrainConfigBuilder
from pyperch.search.adapter import TrainerAdapter
from pyperch.search.factory import SearchFactory

# ------------------------------------------------------------
# 1. Reproducibility
# ------------------------------------------------------------
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

# ------------------------------------------------------------
# 2. Dataset
# ------------------------------------------------------------
X, y = make_classification(
    n_samples=1000,
    n_features=12,
    n_informative=10,
    n_classes=2,
    random_state=seed,
)

X = X.astype(np.float32)
y = y.astype(np.int64)

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=seed
)

train_loader = DataLoader(
    TensorDataset(torch.tensor(X_train), torch.tensor(y_train)),
    batch_size=64,
    shuffle=True,
)

valid_loader = DataLoader(
    TensorDataset(torch.tensor(X_valid), torch.tensor(y_valid)),
    batch_size=64,
)

# ------------------------------------------------------------
# 3. Base training configuration
# ------------------------------------------------------------
base_opt_cfg = OptimizerConfig(
    name="sa",
    t=1.0,
    t_min=0.01,
    step_size=0.05,
    cooling=0.95,
)

base_cfg = TrainConfig(
    device="cpu",
    seed=seed,
    max_epochs=300,
    optimizer="sa",
    optimizer_config=base_opt_cfg,
    optimizer_mode="per_batch",   
    metrics={"train": [Accuracy()], "valid": [Accuracy()]},
)

builder = TrainConfigBuilder(base_cfg)

# ------------------------------------------------------------
# 4. Optuna search space
# ------------------------------------------------------------
def suggest_params(trial):
    return {
        "optimizer_config.t": trial.suggest_float("t", 0.5, 2.0),
        "optimizer_config.t_min": trial.suggest_float("t_min", 0.001, 0.01),
        "optimizer_config.step_size": trial.suggest_float("step_size", 0.025, 0.05),
        "optimizer_config.cooling": trial.suggest_float("cooling", 0.90, 0.999),
        "max_epochs": trial.suggest_int("max_epochs", 10, 200),
    }

strategy = OptunaStrategy(suggest_params)

# ------------------------------------------------------------
# 5. Callbacks
# ------------------------------------------------------------
# Early stopping reduces search runtime
early_stopping = EarlyStopping(
    monitor="valid_loss",   # loss is always minimized
    patience=15,
    min_delta=1e-4,
)

# ------------------------------------------------------------
# 6. Training function (Optuna objective)
# ------------------------------------------------------------
def train_fn(cfg):
    model = SimpleMLP(
        input_dim=12,
        hidden=[32],
        output_dim=2,
        activation="relu",
    )

    loss_fn = nn.CrossEntropyLoss()

    trainer = Trainer(
        model=model,
        config=cfg,
        loss_fn=loss_fn,
    )

    # Ensure default callbacks (e.g., HistoryCallback) are preserved
    if trainer.callbacks is None:
        trainer.callbacks = CallbackList()

    trainer.callbacks.append(early_stopping)

    history = trainer.fit(train_loader, valid_loader)

    # Optuna objective: best validation accuracy achieved
    return max(history["valid_metrics"].get("accuracy", [0.0]))

adapter = TrainerAdapter(builder, strategy, train_fn)

# ------------------------------------------------------------
# 7. Run search
# ------------------------------------------------------------
import optuna

STUDY_NAME = "sa_search_demo"
STORAGE = "sqlite:///sa_demo.db"

# Delete if the study already exists
existing_studies = [s.study_name for s in optuna.get_all_study_summaries(STORAGE)]
if STUDY_NAME in existing_studies:
    optuna.delete_study(
        study_name=STUDY_NAME,
        storage=STORAGE,
    )

search = SearchFactory.optuna_sqlite(
    adapter=adapter,
    study_name="sa_search_demo",
    storage="sqlite:///sa_demo.db",
)

study = search.run(
    n_trials=150,
    n_jobs=4,  # use -1 for all available cores
)

print("Best parameters:", search.best_params)
print("Best score:", search.best_value)
print("Best trial:", search.best_trial.number)

[I 2026-01-14 10:10:26,129] A new study created in RDB with name: sa_search_demo
[I 2026-01-14 10:10:26,516] Trial 0 finished with value: 0.455 and parameters: {'t': 1.2542498596054235, 't_min': 0.006347767831570647, 'step_size': 0.040744746544454494, 'cooling': 0.9648670838607213, 'max_epochs': 199}. Best is trial 0 with value: 0.455.
[I 2026-01-14 10:10:26,656] Trial 1 finished with value: 0.62 and parameters: {'t': 1.1356241058516747, 't_min': 0.007897179868209784, 'step_size': 0.034931345461022015, 'cooling': 0.9004035508327651, 'max_epochs': 118}. Best is trial 1 with value: 0.62.
[I 2026-01-14 10:10:26,737] Trial 2 finished with value: 0.45 and parameters: {'t': 1.7239195888823666, 't_min': 0.003504076532326393, 'step_size': 0.03681489470841783, 'cooling': 0.9762717538431935, 'max_epochs': 19}. Best is trial 1 with value: 0.62.
[I 2026-01-14 10:10:26,797] Trial 3 finished with value: 0.49 and parameters: {'t': 0.8585584569990126, 't_min': 0.0034431046540878076, 'step_size': 0.041

Best parameters: {'t': 1.8652157433496632, 't_min': 0.001316205482456322, 'step_size': 0.048103311881959133, 'cooling': 0.9553752887281385, 'max_epochs': 79}
Best score: 0.66
Best trial: 4
